In [1]:
%matplotlib inline

In [2]:
import uuid
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng

from mesa import Agent, Model

'''
Time in most agent-based models moves in steps, sometimes also called ticks. At each step of the model, one or more of the agents – usually all of them – are activated and take their own step, changing internally and/or interacting with one another or the environment.

The scheduler is a special model component which controls the order in which agents are activated.
The RandomActivation scheduler activates all the agents once per step, in random order.
'''
from mesa.time import RandomActivation


'''
The data collector stores three categories of data: model-level variables, agent-level variables, and tables (which are a catch-all for everything else). Model- and agent-level variables are added to the data collector along with a function for collecting them. Model-level collection functions take a model object as an input, while agent-level collection functions take an agent object as an input. Both then return a value computed from the model or each agent at their current state. When the data collector’s collect method is called, with a model object as its argument, it applies each model-level collection function to the model, and stores the results in a dictionary, associating the current value with the current step of the model. Similarly, the method applies each agent-level collection function to each agent currently in the schedule, associating the resulting value with the step of the model, and the agent’s unique_id
'''
from mesa.datacollection import DataCollector

'''
You usually won’t run a model only once, but multiple times, with fixed parameters to find the overall distributions the model generates, and with varying parameters to analyze how they drive the model’s outputs and behaviors. Instead of needing to write nested for-loops for each model, Mesa provides a BatchRunner class which automates it for you.
'''
from mesa.batchrunner import BatchRunner


from mesa.visualization.ModularVisualization import ModularServer
'''
The basic chart pulls data from the model’s DataCollector, and draws it as a line graph using the Charts.js JavaScript libraries. We instantiate a chart element with a list of series for the chart to track. Each series is defined in a dictionary, and has a Label (which must match the name of a model-level variable collected by the DataCollector) and a Color name. We can also give the chart the name of the DataCollector object in the model.
'''
from mesa.visualization.modules import ChartModule

In [3]:
# De https://github.com/projectmesa/mesa/blob/a44d71ba4b2df0c23c685f6af60cd4b9e5fafffb/examples/wolf_sheep/wolf_sheep/schedule.py

from collections import defaultdict

from mesa.time import RandomActivation


class RandomActivationByBreed(RandomActivation):
    """
    A scheduler which activates each type of agent once per step, in random
    order, with the order reshuffled every step.
    This is equivalent to the NetLogo 'ask breed...' and is generally the
    default behavior for an ABM.
    Assumes that all agents have a step() method.
    """

    def __init__(self, model):
        super().__init__(model)
        self.agents_by_breed = defaultdict(dict)

    def add(self, agent):
        """
        Add an Agent object to the schedule
        Args:
            agent: An Agent to be added to the schedule.
        """

        self._agents[agent.unique_id] = agent
        agent_class = type(agent)
        self.agents_by_breed[agent_class][agent.unique_id] = agent

    def remove(self, agent):
        """
        Remove all instances of a given agent from the schedule.
        """

        del self._agents[agent.unique_id]

        agent_class = type(agent)
        del self.agents_by_breed[agent_class][agent.unique_id]

    def step(self, by_breed=True):
        """
        Executes the step of each agent breed, one at a time, in random order.
        Args:
            by_breed: If True, run all agents of a single breed before running
                      the next one.
        """
        if by_breed:
            for agent_class in self.agents_by_breed:
                self.step_breed(agent_class)
            self.steps += 1
            self.time += 1
        else:
            super().step()

    def step_breed(self, breed):
        """
        Shuffle order and run all agents of a given breed.
        Args:
            breed: Class object of the breed to run.
        """
        agent_keys = list(self.agents_by_breed[breed].keys())
        self.model.random.shuffle(agent_keys)
        for agent_key in agent_keys:
            self.agents_by_breed[breed][agent_key].step()

    def get_breed_count(self, breed_class):
        """
        Returns the current number of agents of certain breed in the queue.
        """
        return len(self.agents_by_breed[breed_class].values())

    def get_breed_list(self, breed_class):
        """
        Returns the current number of agents of certain breed in the queue.
        """
        return self.agents_by_breed[breed_class]

In [4]:
# Model-level data collection function
def compute_gini(model):
    ingresos_tot = [agent.ingreso_total for agent in model.schedule.agents]
    x = sorted(ingresos_tot)
    N = len(model.schedule.agents)
    acc = 0

    for i in range(N):
        acc += (2*(i + 1) - N -1)*x[i]
    G = np.round(acc / (np.mean(x) * N * (N - 1)), 2)

    return G

In [5]:
class Consumidore(Agent):
    ''' An agent with fixed initial wealth.'''
    def __init__(self, unique_id, model, num_empresas, ingreso_inicial, impuesto_ingreso):
        super().__init__(unique_id, model)
        rng = default_rng()

        # Dotaciones iniciales
        # self.theta = 1 / num_empresas
        self.oferta_trabajo_inicial = np.round(rng.uniform(), 2)
        self.ingreso_inicial = ingreso_inicial
        self.oferta_trabajo_disponible = self.oferta_trabajo_inicial
        self.ingreso_total = self.ingreso_inicial
        self.ingreso_disponible = self.ingreso_inicial
        self.impuesto_ingreso = impuesto_ingreso
        self.consumo = 0
        self.max_consumo = self.consumo
        self.delta_cambio_consumo = 0
        self.factor_descanso = np.round(rng.uniform(), 2)
        self.delta_oferta_trabajo = np.round(rng.uniform(0, 1 - self.oferta_trabajo_disponible), 2)
        # Temporal
        self.demanda_trabajo_disponible = np.round(rng.uniform(0, 5), 2)

    def cambio_oferta_trabajo(self):
        camaradas = self.model.schedule.agents

        if len(camaradas) > 1:
            otre = self.random.choice(camaradas)

            if (otre.delta_oferta_trabajo + oferta_trabajo) <= 1:
                self.delta_oferta_trabajo = otre.delta_oferta_trabajo
    
    def vive(self):
        self.ingreso_disponible -= self.model.costo_vida
    
    def jubila(self):
        self.model.schedule.remove(self)

    # Considerar la reprodución (que equivaldrá a que alguien ingresa al mercado laboral)

    def trabaja(self):
        camaradas = self.model.schedule.agents
        
        for i in range(len(camaradas)):
            otre = camaradas[i]
            oferta = self.oferta_trabajo_disponible
    
            if self.oferta_trabajo_disponible > 0 and otre.demanda_trabajo_disponible > 0:
                oferta = np.round(rng.uniform(0, self.oferta_trabajo_disponible), 2)

            salario = rng.integers(self.model.costo_vida, 400) * oferta
            self.oferta_trabajo_disponible -= oferta
            self.ingreso_disponible += salario
            self.ingreso_total += salario
            otre.demanda_trabajo_disponible -= oferta

    def paga_impuestos(self):
        camaradas = self.model.schedule.agents
        impuesto = self.impuesto_ingreso * self.ingreso_disponible
        
        for i in range(len(camaradas)):
            otre = camaradas[i]
            otre.ingreso_disponible += impuesto / len(camaradas)
    
    def consume(self):
        if self.ingreso_disponible > self.model.costo_vida:
            placeres = self.ingreso_disponible - self.model.costo_vida
            self.consumo = placeres

            if (self.consumo > self.max_consumo and self.max_consumo != 0):
                self.delta_cambio_consumo = (self.consumo / self.max_consumo) - 1
                self.max_consumo = self.consumo
            
            self.ingreso_disponible -= placeres
    
    def descansa(self):
        self.oferta_trabajo_disponible = self.factor_descanso * self.oferta_trabajo_disponible
        # Parto del supuesto donde, a mayor consumo, mayor bienestar, y a mayor bienestar, mayor eficiencia laboral. Por tanto, defino el factor consumo como: 
        self.factor_descanso = np.round(rng.uniform(0, self.delta_cambio_consumo), 2)
        # Temporal
        self.demanda_trabajo_disponible = np.round(rng.uniform(0, 5), 2)

    def step(self):
        # Si los ingresos del consumidore son menores a los costos de vivir, se jubila
        if self.ingreso_disponible < self.model.costo_vida:
            self.jubila()
        else:
            self.vive()
            self.trabaja()
            self.paga_impuestos()
            self.consume()
            self.descansa()

class Empresa(Agent):
    ''' An agent with fixed initial wealth.'''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        rng = default_rng()

        # Dotaciones iniciales
        self.demanda_trabajo = 0
        self.capital_inicial = np.round(rng.uniform(0, 5), 2)
        self.costo_capital = np.ro
        self.beneficios = 0
        self.produccion = 0
        self.max_beneficios = 0
        self.costo_trabajo = 0
        self.costo_capital = np.round(rng.uniform(0, 5), 2)
        self.alfa = np.round(rng.uniform(), 2)
        self.num_trabajadores = 0


    def func_prod(trabajo, capital, alfa):
        return np.round((trabajo**alfa)*(capital**(1-alfa)), 2)

    def beneficios(self):
        # Normalizando los precios del bien a 1
        self.beneficios = func_prod(self.demanda_trabajo, self.capital, self.alfa) 

    def produce(self):
        self.beneficios()

    def bancarrota(self):
        self.model.schedule.remove(self)

    def step(self):
        if self.beneficios < 0:
            self.bancarrota()
        
        self.produce()

class EconomiaSocialista(Model):
    '''A model with some number of agents.'''
    def __init__(self, I, J, impuesto_ingreso, costo_vida, ingreso_inicial):
        self.num_consumidores = I
        self.num_empresas = J
        self.schedule = RandomActivationByBreed(self)
        self.running = True
        self.impuesto_ingreso = impuesto_ingreso
        self.ingreso_inicial = ingreso_inicial
        self.costo_vida = costo_vida

        # Create agents
        for i in range(self.num_consumidores):
            a = Consumidore(uuid.uuid4(), self, self.num_empresas, self.ingreso_inicial, self.impuesto_ingreso)
            self.schedule.add(a)
        
        self.datacollector = DataCollector(
            model_reporters={"Gini": compute_gini},  # `compute_gini` defined above
            agent_reporters={"Ingreso total": "ingreso_total"}
        )

    def entrada_mercado_laboral(self):
        nueves_consumidores = self.num_consumidores - len(self.schedule.agents)

        if nueves_consumidores != 0:
            for i in range(nueves_consumidores):
                a = Consumidore(uuid.uuid4(), self, self.num_empresas, self.ingreso_inicial, self.impuesto_ingreso)
                self.schedule.add(a)

    def step(self):
        '''Advance the model by one step.'''
        
        self.entrada_mercado_laboral()
        self.datacollector.collect(self)
        self.schedule.step(by_breed=False)

In [6]:
# ingresos_totales = []
# rng = default_rng()

# for j in range(50):
#     # Run the model
#     model = EconomiaSocialista(rng.integers(100, 1000), 0,  np.round(rng.uniform(), 2), rng.integers(0, 100), rng.integers(0, 100))
#     for i in range(100):
#         model.step()

#     # Store the results
#     for agent in model.schedule.agents:
#         ingresos_totales.append(agent.ingreso_total)
    
#     gini = model.datacollector.get_model_vars_dataframe()
#     gini.plot()
#     plt.show()

# plt.xlabel('Ingreso total')
# plt.ylabel('# de consumidores')
# plt.title('Ingresos totales acumulados por agentes')
# plt.hist(ingresos_totales, bins=range(int(max(ingresos_totales))+1))
# plt.show()

In [7]:
ingresos_totales = []
rng = default_rng()

fixed_params = {
    "J": 0,
}

variable_params = {
    "I": range(100, 1000, 100),
    "impuesto_ingreso": np.round(rng.uniform(0, 1, 10), 2),
    "costo_vida": rng.integers(0, 100, 10),
    "ingreso_inicial": rng.integers(0, 100, 10)
}

# The variables parameters will be invoke along with the fixed parameters allowing for either or both to be honored.
# The BatchRunner won’t collect the data every step of the model, but only at the end of each run.
batch_run = BatchRunner(
    EconomiaSocialista,
    variable_params,
    fixed_params,
    iterations=10,
    max_steps=100,
    model_reporters={"Gini": compute_gini}
)

batch_run.run_all()

run_data = batch_run.get_model_vars_dataframe()
run_data.head()
plt.scatter(run_data.N, run_data.Gini)

# for j in range(10):
#     # Run the model
#     model = EconomiaSocialista(50, 0, 1, 10, 40)
#     for i in range(100):
#         model.step()

#     # Store the results
#     for agent in model.schedule.agents:
#         ingresos_totales.append(agent.ingreso_total)
    
#     gini = model.datacollector.get_model_vars_dataframe()
#     gini.plot()
#     plt.show()

# plt.xlabel('Ingreso total')
# plt.ylabel('# de consumidores')
# plt.title('Ingresos totales acumulados por agentes')
# plt.hist(ingresos_totales, bins=range(int(max(ingresos_totales))+1))
# plt.show()

0it [00:00, ?it/s]


ZeroDivisionError: division by zero

In [ ]:
chart = ChartModule([{"Label": "Gini",
                      "Color": "#18496D"}],
                    data_collector_name='datacollector')

server = ModularServer(EconomiaSocialista,
                       [chart],
                       "Economia Socialista",
                       {
                            "I": 50,
                            "J": 0,
                            "impuesto_ingreso": np.round(rng.uniform(), 2),
                            "costo_vida": rng.integers(0, 100),
                            "ingreso_inicial": rng.integers(0, 100)
                        })
server.port = 8521 # The default
server.launch()